In [1]:
# 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原
# View dataset directory. 
# This directory will be recovered automatically after resetting environment. 
!ls /home/aistudio/data

In [2]:
# 查看工作区文件, 该目录下的变更将会持久保存. 请及时清理不必要的文件, 避免加载过慢.
# View personal work directory. 
# All changes under this directory will be kept even after reset. 
# Please clean unnecessary files in time to speed up environment loading. 
!ls /home/aistudio/work

In [3]:
# 如果需要进行持久化安装, 需要使用持久化路径, 如下方代码示例:
# If a persistence installation is required, 
# you need to use the persistence path as the following: 
!mkdir /home/aistudio/external-libraries
!pip install beautifulsoup4 -t /home/aistudio/external-libraries

mkdir: 无法创建目录"/home/aistudio/external-libraries": 文件已存在
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 7.7 MB/s eta 0:00:00

[notice] A new release of pip available: 22.1.2 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [4]:
# 同时添加如下代码, 这样每次环境(kernel)启动的时候只要运行下方代码即可: 
# Also add the following code, 
# so that every time the environment (kernel) starts, 
# just run the following code: 
import sys 
sys.path.append('/home/aistudio/external-libraries')

In [5]:
# -*- coding:utf-8 -*-
# @FileName  :demo4.py
# @Time      :2023/6/2 15:38
# @Author    :FMY
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import  LogisticRegression
from sklearn.preprocessing import LabelEncoder

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score

data = pd.read_csv('data1.csv',index_col=0)





/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [6]:
postive_user =  data.groupby('user_id').agg({
    'is_order': 'sum'
    })
postive_user = postive_user.reset_index()
postive_user.columns=[
    'user_id',
    'order_sum',
]
postive_user =postive_user[postive_user['order_sum']>0]
data = data[data['user_id'].isin(postive_user['user_id'])]


In [7]:
data = data.drop(['dt','expose_start_time','time_slot1','cat_id','brandsn','goods_clk_sum',
'goods_like_sum','brandsn_addcart_sum','brandsn_order_sum','is_like',
'cat_like_sum','cat_clk_sum','cat_order_sum','brandsn_like_sum','brandsn_clk_sum'],axis=1)
# print('原始数据大小',data.shape)
test = pd.read_excel('./a榜需要预测的uid_5000.xlsx',names=['user_id'])
test = pd.merge(test,data,on=['user_id'])


train = data

train1 = train.drop(['is_order','user_id','goods_id'],axis=1)
test1 = test.drop(['is_order','user_id','goods_id'],axis=1)

In [8]:
y = np.array(train['is_order'].values)
# y = y.reshape((len(y),1))
# print('去除测试集客户后的大小',train.shape)
test1 = np.array(test1)
train1 = np.array(train1)


m1 = int(0.8*len(train1))
x_t = train1[:m1,:]
y_t = y[:m1]
x_t1 = train1[m1:,:]
y_t1 = y[m1:]
#50 ->0.507


In [9]:
randomForest = RandomForestClassifier(n_jobs=-1,n_estimators=70)
model = randomForest.fit(x_t, y_t)
y_pre = model.predict(test1)
# accuracy = f1_score(y_t1,y_pre)
# print('随机森林',accuracy)
# from sklearn.model_selection import GridSearchCV
# param_test1 = {"n_estimators":range(1,101,10)}
# gsearch1 = GridSearchCV(estimator=RandomForestClassifier(n_jobs=-1),param_grid=param_test1,
#                         scoring='f1',cv=10)
# gsearch1.fit(train1, y)


# means =gsearch1.cv_results_['mean_test_score']
# params = gsearch1.cv_results_['params']
# for mean,param in zip(means,params):
#     print("%f  with:   %r" % (mean,param))
 

# log1 = LogisticRegression(max_iter=500)
# model = log1.fit(x_t, y_t)
# y_pre = model.predict(x_t1)
# accuracy = f1_score(y_t1,y_pre)
# print('逻辑回归',accuracy)

# decisiontree = DecisionTreeClassifier()
# model = decisiontree.fit(x_t, y_t)
# y_pre = model.predict(x_t1)
# accuracy = f1_score(y_t1,y_pre)
# print('决策树',accuracy)

res = {'user_id':test['user_id'].values,'goods_id':test['goods_id'].values,'order':y_pre}
df = pd.DataFrame(res)
df = df[df['order']==1]
df = df.drop(columns=['order'])
df.to_csv('u2i.csv',index=False)

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 

In [1]:
# -*- coding:utf-8 -*-
# @FileName  :demo1.py
# @Time      :2023/6/7 16:18
# @Author    :FMY


import pandas as pd
import numpy as np
from datetime import datetime





df = pd.read_csv('allData.csv')
df = df.drop(['dt'], axis=1)
df['date'] = pd.to_datetime(df['expose_start_time'])

date_7_start = '2023-02-25'
date_7_start = datetime.strptime(date_7_start, '%Y-%m-%d')
date_7_end = '2023-03-03'
date_7_end = datetime.strptime(date_7_end, '%Y-%m-%d')
# date_14_start = '2023-02-18'
# date_14_start = datetime.strptime(date_14_start, '%Y-%m-%d')
# date_14_end = '2023-02-24'
# date_14_end = datetime.strptime(date_14_end, '%Y-%m-%d')
# date_21_start = '2023-02-03'
# date_21_start = datetime.strptime(date_21_start, '%Y-%m-%d')
# date_21_end = '2023-02-17'
# date_21_end = datetime.strptime(date_21_end, '%Y-%m-%d')

# 筛选出每个商品距离2023.3.4最近的7天的数据
mask = (df['date'] >= date_7_start) & (df['date'] <= date_7_end)
recent7_data = df.loc[mask]
# # # 筛选出每个商品距离2023.3.4最近的14天的数据
# mask = (df['date'] >= date_14_start) & (df['date'] <= date_14_end)
# recent14_data = df.loc[mask]
# # # 筛选出每个商品距离2023.3.4最近的21天的数据
# mask = (df['date'] >= date_21_start) & (df['date'] <= date_21_end)
# recent21_data = df.loc[mask]

goods7_feat= recent7_data.groupby(['user_id', 'goods_id']).agg({
    'is_clk': ['sum'],
    'is_like': ['sum'],
    'is_addcart': ['sum'],
    'is_order': ['sum']
})
goods7_feat = goods7_feat.reset_index()
goods7_feat.columns = [
    'user_id',
    'goods_id',
    'goods_7clk',
    'goods_7like',
    'goods_7addcart',
    'goods_7order'
]
# goods14_feat= recent14_data.groupby(['user_id', 'goods_id']).agg({
#     'is_clk': ['sum'],
#     'is_like': ['sum'],
#     'is_addcart': ['sum'],
#     'is_order': ['sum']
# })
# goods14_feat = goods14_feat.reset_index()
# goods14_feat.columns = [
#     'user_id',
#     'goods_id',
#     'goods_14clk',
#     'goods_14like',
#     'goods_14addcart',
#     'goods_14order'
# ]
# goods21_feat= recent21_data.groupby(['user_id', 'goods_id']).agg({
#     'is_clk': ['sum'],
#     'is_like': ['sum'],
#     'is_addcart': ['sum'],
#     'is_order': ['sum']
# })
# goods21_feat = goods21_feat.reset_index()
# goods21_feat.columns = [
#     'user_id',
#     'goods_id',
#     'goods_21clk',
#     'goods_21like',
#     'goods_21addcart',
#     'goods_21order'
# ]
cat7_feat= recent7_data.groupby(['user_id','cat_id']).agg({
    'is_clk': ['sum'],
    'is_like': ['sum'],
    'is_addcart': ['sum'],
    'is_order': ['sum']
})
cat7_feat = cat7_feat.reset_index()
cat7_feat.columns = [
    'user_id',
    'cat_id',
    'cat_7clk',
    'cat_7like',
    'cat_7addcart',
    'cat_7order'
]
# cat14_feat= recent14_data.groupby(['user_id','cat_id']).agg({
#     'is_clk': ['sum'],
#     'is_like': ['sum'],
#     'is_addcart': ['sum'],
#     'is_order': ['sum']
# })
# cat14_feat = cat14_feat.reset_index()
# cat14_feat.columns = [
#     'user_id',
#     'cat_id',
#     'cat_14clk',
#     'cat_14like',
#     'cat_14addcart',
#     'cat_14order'
# ]
# cat21_feat= recent21_data.groupby(['user_id','cat_id']).agg({
#     'is_clk': ['sum'],
#     'is_like': ['sum'],
#     'is_addcart': ['sum'],
#     'is_order': ['sum']
# })
# cat21_feat = cat21_feat.reset_index()
# cat21_feat.columns = [
#     'user_id',
#     'cat_id',
#     'cat_21clk',
#     'cat_21like',
#     'cat_21addcart',
#     'cat_21order'
# ]

brand7_feat= recent7_data.groupby(['user_id','brandsn']).agg({
    'is_clk': ['sum'],
    'is_like': ['sum'],
    'is_addcart': ['sum'],
    'is_order': ['sum']
})
brand7_feat = brand7_feat.reset_index()
brand7_feat.columns = [
    'user_id',
    'brandsn',
    'brand_7clk',
    'brand_7like',
    'brand_7addcart',
    'brand_7order'
]
# brand14_feat= recent14_data.groupby(['user_id','brandsn']).agg({
#     'is_clk': ['sum'],
#     'is_like': ['sum'],
#     'is_addcart': ['sum'],
#     'is_order': ['sum']
# })
# brand14_feat = brand14_feat.reset_index()
# brand14_feat.columns = [
#     'user_id',
#     'brandsn',
#     'brand_14clk',
#     'brand_14like',
#     'brand_14addcart',
#     'brand_14order'
# ]
# brand21_feat= recent21_data.groupby(['user_id','brandsn']).agg({
#     'is_clk': ['sum'],
#     'is_like': ['sum'],
#     'is_addcart': ['sum'],
#     'is_order': ['sum']
# })
# brand21_feat = brand21_feat.reset_index()
# brand21_feat.columns = [
#     'user_id',
#     'brandsn',
#     'brand_21clk',
#     'brand_21like',
#     'brand_21addcart',
#     'brand_21order'
# ]
clicked7_data = recent7_data[recent7_data['is_clk'] > 0]
brand7_clk_counts = clicked7_data.groupby('brandsn')['user_id'].nunique()
brand7_clk_counts = brand7_clk_counts.reset_index()
brand7_clk_counts.columns = [
    'brandsn',
    'brand_7clk_counts',
]
like7_data = recent7_data[recent7_data['is_like'] > 0]
brand7_like_counts = like7_data.groupby('brandsn')['user_id'].nunique()
brand7_like_counts = brand7_like_counts.reset_index()
brand7_like_counts.columns = [
    'brandsn',
    'brand_7like_counts',
]
addcart7_data = recent7_data[recent7_data['is_addcart'] > 0]
brand7_addcart_counts = addcart7_data.groupby('brandsn')['user_id'].nunique()
brand7_addcart_counts = brand7_addcart_counts.reset_index()
brand7_addcart_counts.columns = [
    'brandsn',
    'brand_7addcart_counts',
]
order7_data = recent7_data[recent7_data['is_order'] > 0]
brand7_order_counts = order7_data.groupby('brandsn')['user_id'].nunique()
brand7_order_counts = brand7_order_counts.reset_index()
brand7_order_counts.columns = [
    'brandsn',
    'brand_7order_counts',
]

# clicked14_data = recent14_data[recent14_data['is_clk'] > 0]
# brand14_clk_counts = clicked14_data.groupby('brandsn')['user_id'].nunique()
# brand14_clk_counts = brand14_clk_counts.reset_index()
# brand14_clk_counts.columns = [
#     'brandsn',
#     'brand_14clk_counts',
# ]
# like14_data = recent14_data[recent14_data['is_like'] > 0]
# brand14_like_counts = like14_data.groupby('brandsn')['user_id'].nunique()
# brand14_like_counts = brand14_like_counts.reset_index()
# brand14_like_counts.columns = [
#     'brandsn',
#     'brand_14like_counts',
# ]
# addcart14_data = recent14_data[recent14_data['is_addcart'] > 0]
# brand14_addcart_counts = addcart14_data.groupby('brandsn')['user_id'].nunique()
# brand14_addcart_counts = brand14_addcart_counts.reset_index()
# brand14_addcart_counts.columns = [
#     'brandsn',
#     'brand_14addcart_counts',
# ]
# order14_data = recent14_data[recent14_data['is_order'] > 0]
# brand14_order_counts = order14_data.groupby('brandsn')['user_id'].nunique()
# brand14_order_counts = brand14_order_counts.reset_index()
# brand14_order_counts.columns = [
#     'brandsn',
#     'brand_14order_counts',
# ]

# clicked21_data = recent21_data[recent21_data['is_clk'] > 0]
# brand21_clk_counts = clicked21_data.groupby('brandsn')['user_id'].nunique()
# brand21_clk_counts = brand21_clk_counts.reset_index()
# brand21_clk_counts.columns = [
#     'brandsn',
#     'brand_21clk_counts',
# ]
# like21_data = recent21_data[recent21_data['is_like'] > 0]
# brand21_like_counts = like21_data.groupby('brandsn')['user_id'].nunique()
# brand21_like_counts = brand21_like_counts.reset_index()
# brand21_like_counts.columns = [
#     'brandsn',
#     'brand_21like_counts',
# ]
# addcart21_data = recent21_data[recent21_data['is_addcart'] > 0]
# brand21_addcart_counts = addcart21_data.groupby('brandsn')['user_id'].nunique()
# brand21_addcart_counts = brand21_addcart_counts.reset_index()
# brand21_addcart_counts.columns = [
#     'brandsn',
#     'brand_21addcart_counts',
# ]
# order21_data = recent21_data[recent21_data['is_order'] > 0]
# brand21_order_counts = order21_data.groupby('brandsn')['user_id'].nunique()
# brand21_order_counts = brand21_order_counts.reset_index()
# brand21_order_counts.columns = [
#     'brandsn',
#     'brand_21order_counts',
# ]

cat7_clk_counts = clicked7_data.groupby('cat_id')['user_id'].nunique()
cat7_clk_counts = cat7_clk_counts.reset_index()
cat7_clk_counts.columns = [
    'cat_id',
    'cat_7clk_counts',
]

cat7_like_counts = like7_data.groupby('cat_id')['user_id'].nunique()
cat7_like_counts = cat7_like_counts.reset_index()
cat7_like_counts.columns = [
    'cat_id',
    'cat_7like_counts',
]

cat7_addcart_counts = addcart7_data.groupby('cat_id')['user_id'].nunique()
cat7_addcart_counts = cat7_addcart_counts.reset_index()
cat7_addcart_counts.columns = [
    'cat_id',
    'cat_7addcart_counts',
]

cat7_order_counts = order7_data.groupby('cat_id')['user_id'].nunique()
cat7_order_counts = cat7_order_counts.reset_index()
cat7_order_counts.columns = [
    'cat_id',
    'cat_7order_counts',
]

#
# cat14_clk_counts = clicked14_data.groupby('cat_id')['user_id'].nunique()
# cat14_clk_counts = cat14_clk_counts.reset_index()
# cat14_clk_counts.columns = [
#     'cat_id',
#     'cat_14clk_counts',
# ]

# cat14_like_counts = like14_data.groupby('cat_id')['user_id'].nunique()
# cat14_like_counts = cat14_like_counts.reset_index()
# cat14_like_counts.columns = [
#     'cat_id',
#     'cat_14like_counts',
# ]

# cat14_addcart_counts = addcart14_data.groupby('cat_id')['user_id'].nunique()
# cat14_addcart_counts = cat14_addcart_counts.reset_index()
# cat14_addcart_counts.columns = [
#     'cat_id',
#     'cat_14addcart_counts',
# ]

# cat14_order_counts = order14_data.groupby('cat_id')['user_id'].nunique()
# cat14_order_counts = cat14_order_counts.reset_index()
# cat14_order_counts.columns = [
#     'cat_id',
#     'cat_14order_counts',
# ]

#
# cat21_clk_counts = clicked21_data.groupby('cat_id')['user_id'].nunique()
# cat21_clk_counts = cat21_clk_counts.reset_index()
# cat21_clk_counts.columns = [
#     'cat_id',
#     'cat_21clk_counts',
# ]

# cat21_like_counts = like21_data.groupby('cat_id')['user_id'].nunique()
# cat21_like_counts = cat21_like_counts.reset_index()
# cat21_like_counts.columns = [
#     'cat_id',
#     'cat_21like_counts',
# ]

# cat21_addcart_counts = addcart21_data.groupby('cat_id')['user_id'].nunique()
# cat21_addcart_counts = cat21_addcart_counts.reset_index()
# cat21_addcart_counts.columns = [
#     'cat_id',
#     'cat_21addcart_counts',
# ]

# cat21_order_counts = order21_data.groupby('cat_id')['user_id'].nunique()
# cat21_order_counts = cat21_order_counts.reset_index()
# cat21_order_counts.columns = [
#     'cat_id',
#     'cat_21order_counts',
# ]

data = pd.merge(df,goods7_feat,on=['user_id','goods_id'],how='left')
# data = pd.merge(data,goods14_feat,on=['user_id','goods_id'],how='left')
# data = pd.merge(data,goods21_feat,on=['user_id','goods_id'],how='left')
data = pd.merge(data,cat7_feat,on=['user_id','cat_id'],how='left')
# data = pd.merge(data,cat14_feat,on=['user_id','cat_id'],how='left')
# data = pd.merge(data,cat21_feat,on=['user_id','cat_id'],how='left')
data = pd.merge(data,brand7_feat,on=['user_id','brandsn'],how='left')
# data = pd.merge(data,brand14_feat,on=['user_id','brandsn'],how='left')
# data = pd.merge(data,brand21_feat,on=['user_id','brandsn'],how='left')

data = pd.merge(data,brand7_clk_counts,on=['brandsn'],how='left')
data = pd.merge(data,brand7_like_counts,on=['brandsn'],how='left')
data = pd.merge(data,brand7_addcart_counts,on=['brandsn'],how='left')
data = pd.merge(data,brand7_order_counts,on=['brandsn'],how='left')

# data = pd.merge(data,brand14_clk_counts,on=['brandsn'],how='left')
# data = pd.merge(data,brand14_like_counts,on=['brandsn'],how='left')
# data = pd.merge(data,brand14_addcart_counts,on=['brandsn'],how='left')
# data = pd.merge(data,brand14_order_counts,on=['brandsn'],how='left')

# data = pd.merge(data,brand21_clk_counts,on=['brandsn'],how='left')
# data = pd.merge(data,brand21_like_counts,on=['brandsn'],how='left')
# data = pd.merge(data,brand21_addcart_counts,on=['brandsn'],how='left')
# data = pd.merge(data,brand21_order_counts,on=['brandsn'],how='left')

data = pd.merge(data,cat7_clk_counts,on=['cat_id'],how='left')
data = pd.merge(data,cat7_like_counts,on=['cat_id'],how='left')
data = pd.merge(data,cat7_addcart_counts,on=['cat_id'],how='left')
data = pd.merge(data,cat7_order_counts,on=['cat_id'],how='left')

# data = pd.merge(data,cat14_clk_counts,on=['cat_id'],how='left')
# data = pd.merge(data,cat14_like_counts,on=['cat_id'],how='left')
# data = pd.merge(data,cat14_addcart_counts,on=['cat_id'],how='left')
# data = pd.merge(data,cat14_order_counts,on=['cat_id'],how='left')

# data = pd.merge(data,cat21_clk_counts,on=['cat_id'],how='left')
# data = pd.merge(data,cat21_like_counts,on=['cat_id'],how='left')
# data = pd.merge(data,cat21_addcart_counts,on=['cat_id'],how='left')
# data = pd.merge(data,cat21_order_counts,on=['cat_id'],how='left')
data['is_order'] = data['is_order'].apply(lambda x:1 if x>0 else 0)
data.replace(np.nan,0,inplace=True)
data.to_csv('data.csv',index=False)
# print(data.head(10))

In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('data.csv')

In [2]:
from datetime import datetime
date_7_start = '2023-02-25'
date_7_end = '2023-03-03'

# 筛选出每个商品距离2023.3.4最近的7天的数据
mask = (data['date'] >= date_7_start) & (data['date'] <= date_7_end)
data = data.loc[mask]
data = data.drop(['expose_start_time','cat_id','brandsn','date'],axis=1)

In [3]:
from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import  LogisticRegression
from sklearn.preprocessing import LabelEncoder

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score

# postive_user =  data.groupby('user_id').agg({
#     'is_order': 'sum'
#     })
# postive_user = postive_user.reset_index()
# postive_user.columns=[
#     'user_id',
#     'order_sum',
# ]

# postive_user = postive_user[postive_user['order_sum']>0]
# positive_user = data[data['user_id'].isin(postive_user['user_id'])]
# negative_user = data[~data['user_id'].isin(postive_user['user_id'])]


test = pd.read_excel('./a榜需要预测的uid_5000.xlsx',names=['user_id'])
test = pd.merge(test,data,on=['user_id'])
train = data


# train = pd.concat([
#    positive_user,
#    negative_user.sample(int(0.5 * len(negative_user)))
# ], axis=0)
train1 = train[['is_addcart','goods_7addcart','goods_7order','cat_7order','brand_7order']]
test1 = test[['is_addcart','goods_7addcart','goods_7order','cat_7order','brand_7order']]

In [4]:
y = np.array(train['is_order'].values)
test1 = np.array(test1)
train1 = np.array(train1)


m1 = int(0.8*len(train1))
x_t = train1[:m1,:]
y_t = y[:m1]
x_t1 = train1[m1:,:]
y_t1 = y[m1:]
#50 ->0.507

In [5]:
train.columns

Index(['user_id', 'goods_id', 'is_clk', 'is_like', 'is_addcart', 'is_order',
       'goods_7clk', 'goods_7like', 'goods_7addcart', 'goods_7order',
       'cat_7clk', 'cat_7like', 'cat_7addcart', 'cat_7order', 'brand_7clk',
       'brand_7like', 'brand_7addcart', 'brand_7order', 'brand_7clk_counts',
       'brand_7like_counts', 'brand_7addcart_counts', 'brand_7order_counts',
       'cat_7clk_counts', 'cat_7like_counts', 'cat_7addcart_counts',
       'cat_7order_counts'],
      dtype='object')

In [5]:
randomForest = RandomForestClassifier(n_jobs=-1,n_estimators=70)
model = randomForest.fit(x_t, y_t)
y_pre = model.predict(test1)
# accuracy = f1_score(y_t1,y_pre)
# print('随机森林',accuracy)


# import xgboost as xgb
# xgb_classifier = xgb.XGBClassifier(n_jobs=-1,n_estimators=70,max_depth=5,learning_rate=0.1, subsample=0.7, colsample_bytree=0.7)
# xgb_classifier.fit(x_t, y_t)
# y_pre = xgb_classifier.predict(x_t1)
# accuracy = f1_score(y_t1,y_pre)
# print('随机森林',accuracy)

# from sklearn.model_selection import GridSearchCV
# param_test1 = {"n_estimators":range(1,101,10)}
# gsearch1 = GridSearchCV(estimator=RandomForestClassifier(n_jobs=-1),param_grid=param_test1,
#                         scoring='f1',cv=10)
# gsearch1.fit(train1, y)


# means =gsearch1.cv_results_['mean_test_score']
# params = gsearch1.cv_results_['params']
# for mean,param in zip(means,params):
#     print("%f  with:   %r" % (mean,param))
 

# log1 = LogisticRegression(max_iter=500)
# model = log1.fit(x_t, y_t)
# y_pre = model.predict(x_t1)
# accuracy = f1_score(y_t1,y_pre)
# print('逻辑回归',accuracy)

# decisiontree = DecisionTreeClassifier()
# model = decisiontree.fit(x_t, y_t)
# y_pre = model.predict(x_t1)
# # accuracy = f1_score(y_t1,y_pre)
# # print('决策树',accuracy)

res = {'user_id':test['user_id'].values,'goods_id':test['goods_id'].values,'order':y_pre}
df = pd.DataFrame(res)
df = df[df['order']==1]
df = df.drop(columns=['order'])
df.to_csv('u2i.csv',index=False)

In [19]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

rc = {'font.sans-serif': 'SimHei',
      'axes.unicode_minus': False}
corr = train.corr()['is_order']
print(corr)
# plt.figure()
# g1 = sns.heatmap(corr,cmap='cool',annot=True)
# g1.set_title('Correlation coefficient matrix for features to features, features to labels')
# plt.show()


is_clk                   0.061432
is_like                  0.006996
is_addcart               0.210836
is_order                 1.000000
goods_7clk               0.048860
goods_7like              0.008831
goods_7addcart           0.221230
goods_7order             0.837369
cat_7clk                -0.009933
cat_7like               -0.006834
cat_7addcart             0.045665
cat_7order               0.252133
brand_7clk              -0.004443
brand_7like             -0.004881
brand_7addcart           0.038105
brand_7order             0.201963
brand_7clk_counts       -0.014872
brand_7like_counts      -0.014143
brand_7addcart_counts   -0.006075
brand_7order_counts      0.004669
cat_7clk_counts         -0.036895
cat_7like_counts        -0.032912
cat_7addcart_counts     -0.029478
cat_7order_counts       -0.021376
Name: is_order, dtype: float64
